In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re
import numpy as np

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [16]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none"]
models = ["resnet"]
sensors = [0, 1, 2, 3, 4]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000
topks = [2, 3, 4, 5, 6]
topk_empty = []

columns = ['dataset', 'model', 'sensor', 'preprocessing', 'seed', 'kfold', 'accuracy']
accuracies = pd.DataFrame(columns=columns)

jobs_to_repeat = []

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                for sensor in sensors:
                
                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    
                    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+str(sensor)+"_10fold/out/kfold_results.txt"

                    # Check if job was completed
                    if(os.path.exists(acc_file)!=True):
                        print("Results for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+str(sensor)+"_10fold"+" not yet ready!")
                        jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sensor-"+str(sensor))
                        for kfold_tmp in range(10):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, "avg", "-"]
                        results_idx = results_idx+1
                        continue

                    # Check if data was stored safetly
                    num_lines = sum(1 for line in open(acc_file))
                    if(num_lines <= 11):
                        print("Missing data for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+str(sensor)+"_10fold")
                        jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sensor-"+str(sensor))
                        for kfold_tmp in range(10):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, "avg", "-"]
                        results_idx = results_idx+1
                        continue
                    #print(out_path)

                    file1 = open(acc_file, 'r')
                    Lines = file1.readlines()

                    for count, line in enumerate(Lines):
                        #print("Line{}: {}".format(count, line.strip()))
                        if(count>=1 and count <=10):
                            acc_evol=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+str(sensor)+"_10fold/out/log/LSTM+CNN_kf"+str(count-1)+".txt"
                            kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                            #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                            #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, count-1, kf_accuracy]
                            results_idx = results_idx+1
                        if(count==12):
                            kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, sensor, preprocessing, 0, "avg", kf_accuracy]
                            results_idx = results_idx+1

                    file1.close()
                    
accuracies['accuracy'] = accuracies['accuracy'].astype(float)

In [75]:
accuracies

,dataset,model,sensor,preprocessing,seed,kfold,accuracy
0,Exp-in-nops,resnet,0,none,0,0,48.691893
1,Exp-in-nops,resnet,0,none,0,1,48.962161
2,Exp-in-nops,resnet,0,none,0,2,48.794594
3,Exp-in-nops,resnet,0,none,0,3,48.935136
4,Exp-in-nops,resnet,0,none,0,4,48.491892
...,...,...,...,...,...,...,...
215,Exp-out2-nops,resnet,4,none,0,6,40.024325
216,Exp-out2-nops,resnet,4,none,0,7,39.708108
217,Exp-out2-nops,resnet,4,none,0,8,39.983785
218,Exp-out2-nops,resnet,4,none,0,9,39.137837


In [76]:
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_single_sensors_all.csv", index=False)
accuracies.iloc[:, 0:7].loc[accuracies.kfold=='avg'].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_single_sensors_averaged.csv", index=False)

In [34]:
accuracies.loc[(accuracies.kfold=='avg') & (accuracies.model=='resnet') & (accuracies.seed==0)]

,dataset,model,sensor,preprocessing,seed,kfold,accuracy
10,Exp-in-nops,resnet,0,none,0,avg,48.759729
21,Exp-in-nops,resnet,1,none,0,avg,60.224054
32,Exp-in-nops,resnet,2,none,0,avg,64.586217
43,Exp-in-nops,resnet,3,none,0,avg,71.523243
54,Exp-in-nops,resnet,4,none,0,avg,53.144864
65,Exp-out1-nops,resnet,0,none,0,avg,39.320540
76,Exp-out1-nops,resnet,1,none,0,avg,40.524054
87,Exp-out1-nops,resnet,2,none,0,avg,55.502703
98,Exp-out1-nops,resnet,3,none,0,avg,45.000270
109,Exp-out1-nops,resnet,4,none,0,avg,44.344054


In [71]:
accuracies.loc[(accuracies.kfold=='avg') & (accuracies.model=='resnet') & (accuracies.seed==0)].round(2)

,dataset,model,sensor,preprocessing,seed,kfold,accuracy
10,Exp-in-nops,resnet,0,none,0,avg,48.76
21,Exp-in-nops,resnet,1,none,0,avg,60.22
32,Exp-in-nops,resnet,2,none,0,avg,64.59
43,Exp-in-nops,resnet,3,none,0,avg,71.52
54,Exp-in-nops,resnet,4,none,0,avg,53.14
65,Exp-out1-nops,resnet,0,none,0,avg,39.32
76,Exp-out1-nops,resnet,1,none,0,avg,40.52
87,Exp-out1-nops,resnet,2,none,0,avg,55.50
98,Exp-out1-nops,resnet,3,none,0,avg,45.00
109,Exp-out1-nops,resnet,4,none,0,avg,44.34


In [32]:
for dataset in datasets:
    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
    worst = list(accuracies.loc[(accuracies.kfold=='avg') & (accuracies.dataset=="Exp-"+dataset_small)].sort_values(['accuracy'])['sensor'])
    best = list(reversed(worst))
    print("Exp-"+dataset_small+':')
    print("WORST CASE:")
    print(worst)
    print("BEST CASE:")
    print(best)

Exp-in-nops:
WORST CASE:
[0, 4, 1, 2, 3]
BEST CASE:
[3, 2, 1, 4, 0]
Exp-out1-nops:
WORST CASE:
[0, 1, 4, 3, 2]
BEST CASE:
[2, 3, 4, 1, 0]
Exp-out1-random:
WORST CASE:
[0, 1, 3, 4, 2]
BEST CASE:
[2, 4, 3, 1, 0]
Exp-out2-nops:
WORST CASE:
[4, 0, 1, 2, 3]
BEST CASE:
[3, 2, 1, 0, 4]
